In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D,GlobalAveragePooling2D,Dense,Softmax,Flatten,MaxPooling2D,Dropout,Activation, Lambda, concatenate,BatchNormalization
from keras.datasets import cifar100, cifar10, mnist
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import kullback_leibler_divergence
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import categorical_accuracy
from keras import regularizers
from keras import optimizers
from keras import backend as K
import tensorflow_addons as tfa
import tensorflow_probability as tfp
import torch
import random
import gc
#import seaborn as sns
import h5py
import sys

#import os
import pandas as pd
import keras
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Input, Add, BatchNormalization, Activation
from keras.activations import relu, softmax
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
tf.config.run_functions_eagerly(True)
tf.config.experimental_run_functions_eagerly(True)

C:\Users\hyeli\Anaconda3\envs\hlnam\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20210111). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [2]:
num_classes = 10
input_shape = (32, 32, 3)

# Load the train and test data splits
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Display shapes of train and test datasets
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [3]:

def random_resize_crop(image):
    start_x=random.randint(1,32)
    start_y=random.randint(1,32)
    height=random.randint(1,32)
    length=random.randint(1,32)

    for i in range(start_x, min(31, start_x+length)):
        for j in range(start_y, min(31, start_y+height)):
            image[i][j]=0
    return image
    
    

def color_jitter(x):
    s=1.0
    x = tf.image.random_brightness(x, max_delta=0.8*s)
    x = tf.image.random_contrast(x, lower=1 - 0.8 * s, upper=1 + 0.8 * s)
    x = tf.image.random_saturation(x, lower=1 - 0.8 * s, upper=1 + 0.8 * s)
    x = tf.image.random_hue(x, max_delta=0.2 * s)
    # x = tf.clip_by_value(x, 0, 1)
    return x


def color_drop(x):
    x = tf.image.rgb_to_grayscale(x)
    x = tf.tile(x, [1, 1, 3])
    return x




def data_augmentation_2(image):

    number = tf.random.uniform(shape=[], minval=1, maxval=100)
    if (number<=40):
        image = random_resize_crop(image)
    elif (number<=70):
        image= color_jitter(image)
    else:
        image= color_drop(image)
    return image





In [4]:
## data
import random

x_train_aug=np.zeros_like(x_train)
y_train_aug= y_train.copy()
y_test_aug= y_test.copy()
x_test_aug=np.zeros_like(x_test)


# Train images process
for i in range(x_train.shape[0]):
    x= x_train[i].copy()
    x= data_augmentation_2(x)              #plt.imshow(x)
    x_train_aug[i]=x
x_train_aug= tf.image.convert_image_dtype(x_train_aug, dtype=tf.float32, saturate=False, name=None)


# Test images process
for i in range(x_test.shape[0]):
    x= x_test[i].copy()
    x= data_augmentation_2(x)              #plt.imshow(x)
    x_test_aug[i]=x
x_test_aug= tf.image.convert_image_dtype(x_test_aug, dtype=tf.float32, saturate=False, name=None)



n_train = x_train_aug.shape[0]
shuffle_idx = np.arange(n_train)
np.random.shuffle(shuffle_idx)

x_train_shf = x_train[shuffle_idx][:n_train]
y_train_shf = y_train[shuffle_idx][:n_train]
y_train_shf= to_categorical(y_train_shf, num_classes)
y_test= to_categorical(y_test, num_classes)




print(f'Contrastive learning__ x_train_aug: {x_train_aug.shape}     y_train: {y_train_aug.shape}')
print(f'Contrastive evaluate__ x_test_aug: {x_test_aug.shape}     y_test: {y_test_aug.shape}')
print('\n')
print(f'Supervised learning__ x_train_shf: {x_train_shf.shape}     y_train_shf: {y_train_shf.shape}')
print(f'Supervised evaluate__ x_test: {x_test.shape}     y_test: {y_test.shape}')



Contrastive learning__ x_train_aug: (50000, 32, 32, 3)     y_train: (50000, 1)
Contrastive evaluate__ x_test_aug: (10000, 32, 32, 3)     y_test: (10000, 1)


Supervised learning__ x_train_shf: (50000, 32, 32, 3)     y_train_shf: (50000, 10)
Supervised evaluate__ x_test: (10000, 32, 32, 3)     y_test: (10000, 10)


In [67]:
# Parameter

learning_rate = 0.0001   #0.001
batch_size = 100
hidden_units = 512
projection_units = 128
num_epochs_con = 5
num_epochs_sup = 5
dropout_rate = 0.5


#contrastive: Projection
#Supervised: Classifier

In [13]:
def create_encoder():

    inputs = keras.Input(shape=input_shape)

    x= Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x= BatchNormalization()(x)
    x= Dropout(0.3)(x)
    x= Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(x)
    x= BatchNormalization()(x)
    x= MaxPooling2D(pool_size=(2, 2))(x)
    x= Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(x)
    x= BatchNormalization()(x)
    x= Dropout(0.4)(x)
    x= Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(x)
    x= BatchNormalization()(x)
    x= MaxPooling2D(pool_size=(2, 2))(x)
    
    model = keras.Model(inputs=inputs, outputs=x)
    
    return model





def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features= Flatten()(features)
    features= layers.Dropout(dropout_rate)(features)
    features=layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        #optimizer=keras.optimizers.SGD(learning_rate= learning_rate, decay= 1e-6, momentum=0.9, nesterov=True),
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.CategoricalCrossentropy(),
        #metrics=[keras.metrics.CategoricalAccuracy()],
        metrics=['accuracy']
    )
    return model


In [15]:
def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features= Flatten()(features)
    features= layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    return model

In [64]:
def pdist_euclidean(A):
    # Euclidean pdist
    r = tf.reduce_sum(A*A, 1)

    # turn r into column vector
    r = tf.reshape(r, [-1, 1])
    D = r - 2*tf.matmul(A, tf.transpose(A)) + tf.transpose(r)

    return tf.sqrt(D)


def square_to_vec(D):
    '''Convert a squared form pdist matrix to vector form.
    '''
    n = D.shape[0]
    triu_idx = np.triu_indices(n, k=1)
    d_vec = tf.gather_nd(D, list(zip(triu_idx[0], triu_idx[1])))
    return d_vec


def get_contrast_batch_labels(y):
    '''
    Make contrast labels by taking all the pairwise in y
    y: tensor with shape: (batch_size, )
    returns:
        tensor with shape: (batch_size * (batch_size-1) // 2, )
    '''
    y_col_vec = tf.reshape(tf.cast(y, tf.float32), [-1, 1])
    D_y = pdist_euclidean(y_col_vec)
    d_y = square_to_vec(D_y)
    y_contrasts = tf.cast(d_y == 0, tf.int32)
    return y_contrasts

 
    
    
    
    
    
    
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)
    
    
    
class max_margin_contrastive_loss_euclidean(keras.losses.Loss):
    def __init__(self, margin=1.0, name=None):
        super(max_margin_contrastive_loss_euclidean, self).__init__(name=name)
        self.margin=margin
    def __call__(self,labels, feature_vectors, sample_weight=None):
        feature_vectors=tf.math.l2_normalize(feature_vectors, axis=1)
        D= pdist_euclidean(feature_vectors)
        d_vec= square_to_vec(D)
        y_contrasts= get_contrast_batch_labels(labels)
        loss= tfa.losses.contrastive_loss(y_contrasts, d_vec, margin=margin)
        loss= tf.reduce_mean(loss)
        return loss
    


    
class max_margin_contrastive_loss_cosine(keras.losses.Loss):
    def __init__(self, margin=1.0, name=None):
        super(max_margin_contrastive_loss_cosine, self).__init__(name=name)
        self.margin=margin
    def __call__(self, y_true, y_pred, sample_weight=None):
        y_pred=tf.math.l2_normalize(y_pred, axis=1)
        D= 1- tf.matmul(y_pred, y_pred, transpose_b=True)
        d_vec= square_to_vec(D)
        y_contrasts= get_contrast_batch_labels(y_true)
        loss= tfa.losses.contrastive_loss(y_contrasts, d_vec, margin=margin)
        loss= tf.reduce_mean(loss)
        return loss
    
    
    
    
class multiclass_npairs_loss(keras.losses.Loss):   #SupervisedContrastiveLoss랑 똑같은데 temp로 나눈거
    def __init__(self, name=None):
        super(multiclass_npairs_loss, self).__init__(name=name)
    def __call__(self, y_true, y_pred, sample_weight=None):
        y_pred=tf.math.l2_normalize(y_pred, axis=1)
        S= tf.matmul(y_pred, y_pred, transpose_b= True)
        #loss= tfa.losses.npairs_loss(y_true, S)
        loss= tfa.losses.npairs_loss(tf.squeeze(y_true), S)
        return loss

    
    
    
class triplet_loss(keras.losses.Loss):
    def __init__(self, margin=1.0, kind='hard', name=None):
        super(triplet_loss, self).__init__(name=name)
        self.margin=margin
        self.kind= kind
    def __call__(self, y_true, y_pred, sample_weight=None):
        if kind=='hard':
            loss= tfa.losses.triplet_hard_loss(y_true, y_pred, margin=margin, soft=False)
        elif kind=='soft':
            loss= tfa.losses.triplet_hard_loss(y_true, y_pred, margin=margin, soft=True)
        elif kind=='semihard':
            loss= tfa.losses.triplet_semihard_loss(y_true, y_pred, margin=margin)
        return loss

    
    
    
class supervised_nt_xent_loss(keras.losses.Loss):
    def __init__(self,temperature=0.5, name=None):
        super(supervised_nt_xent_loss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self,  labels, feature_vectors, sample_weight=None):
        base_temperature = 1

        feature_vectors=tf.math.l2_normalize(feature_vectors, axis=1)
        batch_size= tf.shape(feature_vectors)[0]
        contrast_count=1
        anchor_count=contrast_count
        labels= tf.expand_dims(labels,-1)

        # mask는 [batch_size, batch_size] dim이고, mask[i,j]=1 if i랑 j의 class 같으면
        mask= tf.cast(tf.equal(labels, tf.transpose(labels)), tf.float32)
        mask= tf.reshape(mask, (mask.shape[0], mask.shape[0]))#
        anchor_dot_contrast= tf.divide(tf.matmul(feature_vectors, tf.transpose(feature_vectors)), self.temperature)

        # for numerical stability
        logits_max= tf.reduce_max(anchor_dot_contrast, axis=1, keepdims=True)
        logits= anchor_dot_contrast - logits_max

        # tile mask 
        logits_mask= tf.ones_like(mask) - tf.eye(batch_size)  #eye는 가운데 eigen
        mask= mask*logits_mask

        # compute log_prob
        exp_logits= tf.exp(logits)*logits_mask
        log_prob= logits - tf.math.log(tf.reduce_sum(exp_logits, axis=1, keepdims=True))

        # compute mean of log-likelihood over positive 
        mask_sum = tf.reduce_sum(mask, axis=1)
        mean_log_prob_pos= tf.reduce_sum(mask*log_prob, axis=1)[mask_sum>0] / mask_sum[mask_sum>0]

        #loss
        loss= -(self.temperature/base_temperature)*mean_log_prob_pos
        loss= tf.reduce_mean(loss)
        
        return loss




In [69]:
# Contrastive learning

encoder = create_encoder()
margin=1.0
temperature=0.2
kind='hard'
#custom_loss=SupervisedContrastiveLoss(temperature)      #낫밷 .adam opti가 더 나음 sgd보다
#custom_loss=max_margin_contrastive_loss_euclidean(margin)  #nan  #l2norm 하니까 나옴 근데 10%
#custom_loss=max_margin_contrastive_loss_cosine(margin)  #supervised 가 10-> batchsize 줄여서 25
#custom_loss=multiclass_npairs_loss()   #ㄱㅊ
#custom_loss=triplet_loss(margin, kind)   #supervised 가 14%
custom_loss=supervised_nt_xent_loss(temperature)    #nan   #l2norm 하니까 나옴  #lr 0.0001하니까 43퍼나옴



encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=custom_loss,
)

encoder_with_projection_head.summary()

history_p = encoder_with_projection_head.fit(
    x=x_train_aug, y=y_train_aug, batch_size=batch_size, epochs=num_epochs_con)



Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_53 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
model_17 (Functional)        (None, 8, 8, 128)         261696    
_________________________________________________________________
flatten_33 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dropout_86 (Dropout)         (None, 8192)              0         
_________________________________________________________________
dense_53 (Dense)             (None, 128)               1048704   
Total params: 1,310,400
Trainable params: 1,309,632
Non-trainable params: 768
_________________________________________________________________
Epoch 1/5
500/500 [==============================] - 20s 39ms/step - loss: 1.0423
Epoc

In [68]:
# Supervised Learning

classifier = create_classifier(encoder_with_projection_head, trainable=False)
classifier.summary()


history_c = classifier.fit(x=x_train_shf, y=y_train_shf, batch_size=batch_size, epochs=num_epochs_sup)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


Model: "cifar10-classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_51 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
cifar-encoder_with_projectio (None, 128)               1310400   
_________________________________________________________________
flatten_32 (Flatten)         (None, 128)               0         
_________________________________________________________________
dropout_82 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 512)               66048     
_________________________________________________________________
dropout_83 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 10)         

In [34]:
# No contrastive, just supervised

encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()

history = classifier.fit(x=x_train_shf, y=y_train_shf, batch_size=batch_size, epochs=num_epochs_con+num_epochs_sup)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Model: "cifar10-classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
model_8 (Functional)         (None, 8, 8, 128)         261696    
_________________________________________________________________
flatten_15 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 8192)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               4194816   
_________________________________________________________________
dropout_42 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 128)        

KeyboardInterrupt: 